In [36]:
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.decomposition import TruncatedSVD

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import twint

In [38]:
import re
import string

In [3]:
import nest_asyncio

In [4]:
nest_asyncio.apply()

# Uncomment below cell!!!

In [ ]:
# # Configure
# c = twint.Config()
# c.Username = "voguemagazine"
# c.Since = "2020-01-01"
# c.Until = "2020-07-01"
# c.Store_csv = True
# c.Output = "voguemagazine"

# # Run
# twint.run.Search(c)

In [ ]:
#c

In [5]:
df_all_info_tweets = pd.read_csv('voguemagazine/tweets.csv')

In [6]:
df_all_info_tweets

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1277748739491454977,1277748739491454977,2020-06-29 16:40:13 PDT,2020-06-29,16:40:13,-800,136361303,voguemagazine,Vogue Magazine,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1277741076506427392,1277741076506427392,2020-06-29 16:09:46 PDT,2020-06-29,16:09:46,-800,136361303,voguemagazine,Vogue Magazine,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1277735264606195712,1277735264606195712,2020-06-29 15:46:40 PDT,2020-06-29,15:46:40,-800,136361303,voguemagazine,Vogue Magazine,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1277728986907455489,1277728986907455489,2020-06-29 15:21:44 PDT,2020-06-29,15:21:44,-800,136361303,voguemagazine,Vogue Magazine,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1277721201675309056,1277721201675309056,2020-06-29 14:50:48 PDT,2020-06-29,14:50:48,-800,136361303,voguemagazine,Vogue Magazine,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9445,1212199987012153345,1212199987012153345,2019-12-31 18:32:53 PST,2019-12-31,18:32:53,-800,136361303,voguemagazine,Vogue Magazine,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
9446,1212192155973902337,1212192155973902337,2019-12-31 18:01:46 PST,2019-12-31,18:01:46,-800,136361303,voguemagazine,Vogue Magazine,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
9447,1212184513490640896,1212184513490640896,2019-12-31 17:31:23 PST,2019-12-31,17:31:23,-800,136361303,voguemagazine,Vogue Magazine,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
9448,1212176343871475712,1212176343871475712,2019-12-31 16:58:56 PST,2019-12-31,16:58:56,-800,136361303,voguemagazine,Vogue Magazine,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


## Data cleaning

In [7]:
df_all_info_tweets.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

In [8]:
# copy of database with date and tweet
df= df_all_info_tweets.loc[:,['date', 'tweet']]

In [9]:
df.tweet.iloc[0] 

'If you’re looking to revamp your yoga or home-workout routine, Vogue Beauty editor @akiliking_ shares why she considers this the perfect mat.  https://t.co/OF9EkbSizw'

Tweet has link at the end, I'll delete it

In [10]:
for i, tweet in enumerate(df['tweet']):
    df['tweet'].iloc[i] = tweet.rsplit(' ', 1)[0]

In [11]:
df.tweet.iloc[0]

'If you’re looking to revamp your yoga or home-workout routine, Vogue Beauty editor @akiliking_ shares why she considers this the perfect mat. '

## Text preprocessing and Document-Term Matrix

In [12]:
# Text preprocessing steps - remove numbers, captial letters and punctuation
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

df['tweet'] = df.tweet.map(alphanumeric).map(punc_lower)
df.head()

,date,tweet
0,2020-06-29,if you’re looking to revamp your yoga or home ...
1,2020-06-29,from soothing meditations to supplements and t...
2,2020-06-29,warriors in the garden’s warriorsitg livia ...
3,2020-06-29,fast fashion may be getting cheaper while luxu...
4,2020-06-29,all of the apps classes and workouts that ca...


In [13]:
X = df.tweet

In [41]:
# The first document-term matrix has default Count Vectorizer values - counts of unigrams

vectorizer = CountVectorizer(stop_words='english')

doc_word = vectorizer.fit_transform(X)

pd.DataFrame(X_cv1.toarray(), columns=cv1.get_feature_names()).head()

,aaronsorkin,ababa,abandoned,abba,abbey,abby,abdi,abe,aberfan,abetting,...,zoom,zoomtopia,zoë,zukin,zuri,zwirner,àcheval,álvarez,ísis,ýrúrarí
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
doc_word.shape

(9450, 11319)

In [33]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

## Non-Negative Matrix Factorization (NMF)

In [52]:
nmf_model = NMF(2)
doc_topic = nmf_model.fit_transform(doc_word)

The W matrix shows us the 2 resulting topics, and the terms that are associated with each topic. In this case:
- Component 1 (topic 1) seems to be about pandemic in NY
- Component 2 (topic 2) seems to be about fashion and beauty

In [53]:
topic_word = pd.DataFrame(nmf_model.components_.round(3),
             index = ["component_1","component_2"],
             columns = vectorizer.get_feature_names())
topic_word

,aaronsorkin,ababa,abandoned,abba,abbey,abby,abdi,abe,aberfan,abetting,...,zoom,zoomtopia,zoë,zukin,zuri,zwirner,àcheval,álvarez,ísis,ýrúrarí
component_1,0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.000,0.000,0.005,...,0.025,0.0,0.000,0.000,0.000,0.006,0.0,0.0,0.000,0.0
component_2,0.001,0.0,0.001,0.0,0.011,0.0,0.007,0.001,0.001,0.000,...,0.150,0.0,0.002,0.001,0.004,0.000,0.0,0.0,0.009,0.0


In [54]:
display_topics(nmf_model, vectorizer.get_feature_names(), 10)


Topic  0
new, york, city, year, need, series, pandemic, life, coronavirus, video

Topic  1
fashion, vogue, week, look, best, time, home, https, shares, red


The H matrix shows us the documents we started with, and how each document is made up of the 2 resulting topics. In this case:
- The first document seems to be about pandemic in NY
- The last document seems to be about fashion and beauty
- Everything in between is a combination of the two

In [59]:
H = pd.DataFrame(doc_topic.round(5),
                 index = df.tweet,
             columns = ["component_1","component_2" ])
H

,component_1,component_2
tweet,,
if you’re looking to revamp your yoga or home workout routine vogue beauty editor akiliking shares why she considers this the perfect mat,0.00000,0.20394
from soothing meditations to supplements and teas here are approaches to get a sounder night’s sleep,0.00000,0.00775
warriors in the garden’s warriorsitg livia rose johnson sees hair as an invaluable means of self expression,0.00480,0.02026
fast fashion may be getting cheaper while luxury gets more expensive–but it comes down to ethical labor fabric quality and profit margins,0.00000,0.11553
all of the apps classes and workouts that can help you maintain a super active lifestyle from home,0.00299,0.03952
...,...,...
vogue staffers on the movies that affected them most this decade,0.00000,0.08332
treat yourself in,0.00018,0.00018
in dualipa had a breakout year not only with her music but also her style,0.01108,0.03518


## Non-Negative Matrix Factorization (NMF) Second version

In [66]:
vectorizer = CountVectorizer(stop_words = 'english')
doc_word = vectorizer.fit_transform(df.tweet)
doc_word.shape

(9450, 11319)

In [67]:
nmf_model = NMF(2)
doc_topic = nmf_model.fit_transform(doc_word)
doc_topic.shape

(9450, 2)

The **doc_topic** matrix shows us the documents we started with, and how each document is made up of the 2 resulting topics. We don't know yet what the topics are.

In [68]:
topic_word = nmf_model.components_
topic_word.shape

(2, 11319)

The **topic_word** matrix shows us the 2 resulting topics, and the terms that are associated with each topic. By looking at the words below, we an figure out what the topics are.

In [69]:
words = vectorizer.get_feature_names()
t = nmf_model.components_.argsort(axis=1)[:,-1:-7:-1]
topic_words = [[words[e] for e in l] for l in t]
topic_words

[['new', 'york', 'city', 'year', 'need', 'series'],
 ['fashion', 'vogue', 'week', 'look', 'best', 'time']]

In [70]:
doc_topic

array([[0.00000000e+00, 2.03936932e-01],
       [0.00000000e+00, 7.74698061e-03],
       [4.80156692e-03, 2.02554865e-02],
       ...,
       [1.10819075e-02, 3.51791803e-02],
       [0.00000000e+00, 3.79059699e-05],
       [0.00000000e+00, 6.79632295e-02]])

In [71]:
doc_topic.argmax(axis=1)

array([1, 1, 1, ..., 1, 1, 1])

In [72]:
df['topic'] = doc_topic.argmax(axis=1)

In [73]:
df

,date,tweet,topic
0,2020-06-29,if you’re looking to revamp your yoga or home ...,1
1,2020-06-29,from soothing meditations to supplements and t...,1
2,2020-06-29,warriors in the garden’s warriorsitg livia ...,1
3,2020-06-29,fast fashion may be getting cheaper while luxu...,1
4,2020-06-29,all of the apps classes and workouts that ca...,1
...,...,...,...
9445,2019-12-31,vogue staffers on the movies that affected the...,1
9446,2019-12-31,treat yourself in,1
9447,2019-12-31,in dualipa had a breakout year not only wi...,1
9448,2019-12-31,do you hate hearing recordings of yourself yo...,1


In [74]:
from textblob import TextBlob

In [75]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

In [78]:
sid_obj = SentimentIntensityAnalyzer()
sentiment = []
for tweet in df.tweet:
    sentiment.append(sid_obj.polarity_scores(tweet))

In [79]:
sentiment

[{'neg': 0.0, 'neu': 0.673, 'pos': 0.327, 'compound': 0.8658},
 {'neg': 0.0, 'neu': 0.795, 'pos': 0.205, 'compound': 0.3818},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 0.763, 'pos': 0.237, 'compound': 0.7351},
 {'neg': 0.0, 'neu': 0.617, 'pos': 0.383, 'compound': 0.8519},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 0.919, 'pos': 0.081, 'compound': 0.4404},
 {'neg': 0.0, 'neu': 0.932, 'pos': 0.068, 'compound': 0.3612},
 {'neg': 0.042, 'neu': 0.637, 'pos': 0.32, 'compound': 0.9246},
 {'neg': 0.066, 'neu': 0.934, 'pos': 0.0, 'compound': -0.4019},
 {'neg': 0.0, 'neu': 0.846, 'pos': 0.154, 'compound': 0.7351},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.149, 'neu': 0.678, 'pos': 0.173, 'compound': 0.0649},
 {'neg': 0.181, 'neu': 0.778, 'pos': 0.041, 'compound': -0.7579},
 {'neg': 0.127, 'neu': 0.636, 'pos': 0.237, 'compound': 0.7184},
 {'neg': 0.085, 'neu': 0.854, 'pos': 0.06, 'compound': -0.0516},
 {

In [80]:
pd.concat([df,pd.DataFrame(sentiment)], axis=1)

,date,tweet,topic,neg,neu,pos,compound
0,2020-06-29,if you’re looking to revamp your yoga or home ...,1,0.000,0.673,0.327,0.8658
1,2020-06-29,from soothing meditations to supplements and t...,1,0.000,0.795,0.205,0.3818
2,2020-06-29,warriors in the garden’s warriorsitg livia ...,1,0.000,1.000,0.000,0.0000
3,2020-06-29,fast fashion may be getting cheaper while luxu...,1,0.000,0.763,0.237,0.7351
4,2020-06-29,all of the apps classes and workouts that ca...,1,0.000,0.617,0.383,0.8519
...,...,...,...,...,...,...,...
9445,2019-12-31,vogue staffers on the movies that affected the...,1,0.138,0.862,0.000,-0.1531
9446,2019-12-31,treat yourself in,1,0.000,0.426,0.574,0.4019
9447,2019-12-31,in dualipa had a breakout year not only wi...,1,0.000,1.000,0.000,0.0000
9448,2019-12-31,do you hate hearing recordings of yourself yo...,1,0.256,0.623,0.120,-0.4515


## Latent Semantic Analysis (LSA)

In [60]:
# Acronynms: Latent Semantic Analysis (LSA) is just another name for 
#  Signular Value Decomposition (SVD) applied to Natural Language Processing (NLP)
lsa = TruncatedSVD(2)
doc_topic = lsa.fit_transform(doc_word)
lsa.explained_variance_ratio_

array([0.00935391, 0.01000748])

The U matrix shows us the 2 resulting topics, and the terms that are associated with each topic. In this case:
- Component 1 (topic 1) seems to be about fashion
- Component 2 (topic 2) seems to be about entertainment

In [61]:
topic_word = pd.DataFrame(lsa.components_.round(3),
             index = ["component_1","component_2"],
             columns = vectorizer.get_feature_names())
topic_word

,aaronsorkin,ababa,abandoned,abba,abbey,abby,abdi,abe,aberfan,abetting,...,zoom,zoomtopia,zoë,zukin,zuri,zwirner,àcheval,álvarez,ísis,ýrúrarí
component_1,0.0,0.0,0.0,0.0,0.001,0.0,0.001,0.0,0.0,0.000,...,0.022,0.0,0.0,0.0,0.000,0.001,0.0,0.0,0.001,0.0
component_2,-0.0,0.0,-0.0,-0.0,-0.002,-0.0,-0.001,-0.0,-0.0,0.001,...,-0.019,-0.0,-0.0,-0.0,-0.001,0.001,-0.0,-0.0,-0.002,-0.0


In [62]:
display_topics(lsa, vectorizer.get_feature_names(), 5)


Topic  0
new, fashion, vogue, york, look

Topic  1
new, york, city, netflix, book


The Vt matrix shows us the documents we started with, and how each document is made up of the 2 resulting topics. In this case:
- The first four documents seem to be about sports
- The last three documents seem to be about music

In [64]:
Vt = pd.DataFrame(doc_topic.round(5),
             index = X,
             columns = ["component_1","component_2" ])
Vt

,component_1,component_2
tweet,,
if you’re looking to revamp your yoga or home workout routine vogue beauty editor akiliking shares why she considers this the perfect mat,0.70470,-0.79733
from soothing meditations to supplements and teas here are approaches to get a sounder night’s sleep,0.02795,-0.02902
warriors in the garden’s warriorsitg livia rose johnson sees hair as an invaluable means of self expression,0.10265,-0.05051
fast fashion may be getting cheaper while luxury gets more expensive–but it comes down to ethical labor fabric quality and profit margins,0.39555,-0.42965
all of the apps classes and workouts that can help you maintain a super active lifestyle from home,0.16521,-0.12289
...,...,...
vogue staffers on the movies that affected them most this decade,0.28028,-0.33948
treat yourself in,0.00165,0.00010
in dualipa had a breakout year not only with her music but also her style,0.19058,-0.07821
